## Hans Tausen iskappe in Greenland


first we need to import some libraries

In [1]:
# general libraries

import os
import urllib
import numpy as np
import matplotlib.pyplot as plt

from scipy import ndimage
from PIL import Image

In [4]:
# import functions from specific libraries
from eratosthenes.generic.mapping_io import \
    read_geo_image, read_geo_info, make_geo_im
from eratosthenes.generic.mapping_tools import \
    pix_centers
from eratosthenes.generic.handler_im import get_grad_filters
from eratosthenes.generic.gis_tools import get_mask_boundary
from eratosthenes.preprocessing.image_transforms import \
    gamma_adjustment, mat_to_gray
from eratosthenes.preprocessing.shadow_filters import \
    anistropic_diffusion_scalar
from eratosthenes.postprocessing.displacement_filters import \
    local_infilling_filter
from eratosthenes.postprocessing.terrain_tools import d8_flow
from eratosthenes.presentation.terrain_tools import curvature_enhanced_shading
from eratosthenes.presentation.image_io import output_image, output_mask


The data is situated on a drive, but these can be downloaded to a local drive. Hence the following code sniplet looks if such data is already available, if not it is downloaded.

In [6]:
# admin
dat_dir = '/Users/Alten005/icedivide'
dat_url = 'https://surfdrive.surf.nl/files/index.php/s/qebmYP7idEW89O0'

vel_name = 'greenlan_iv_100m_s1_insar_s20181201_e20210301_v1_'
v_x_name, v_y_name = 'east_velo.tif', 'north_velo.tif'
v_x_std, v_y_std = 'east_std.tif', 'north_std.tif'
v_count = 'count.tif'
rgi_name = 'glacier_ids.tif'
dem_name = 'arcticdem_100m.tif'

# do downloading if files are not present
if not os.path.exists(os.path.join(dat_dir, rgi_name)):
    file_grab = urllib.URLopener()
    print('busy downloading files')
    # dowload velocity data
    file_grab.retrieve(os.path.join(dat_url, vel_name + v_x_name),
                       os.path.join(dat_dir, vel_name + v_x_name))
    file_grab.retrieve(os.path.join(dat_url, vel_name + v_y_name),
                       os.path.join(dat_dir, vel_name + v_y_name))
    file_grab.retrieve(os.path.join(dat_url, vel_name + v_x_std),
                       os.path.join(dat_dir, vel_name + v_x_std))
    file_grab.retrieve(os.path.join(dat_url, vel_name + v_y_std),
                       os.path.join(dat_dir, vel_name + v_y_std))
    file_grab.retrieve(os.path.join(dat_url, vel_name + v_count),
                       os.path.join(dat_dir, vel_name + v_count))
    # download auxillary data
    file_grab.retrieve(os.path.join(dat_url, rgi_name),
                       os.path.join(dat_dir, rgi_name))
    file_grab.retrieve(os.path.join(dat_url, dem_name),
                       os.path.join(dat_dir, dem_name))
    print('files dowloaded')
else:
    print('files are already present')

files are already present


import rasters with glacier polygons (RGI) and elevation (Z)

In [7]:
spatialRef, geoTransform,_,_,_,_ = read_geo_info(
    os.path.join(dat_dir, rgi_name))
RGI = read_geo_image(os.path.join(dat_dir, rgi_name))[0]
Z = read_geo_image(os.path.join(dat_dir, dem_name))[0]
RGI[RGI==1], Z[Z==-9999] = 0, np.nan
X,Y = pix_centers(geoTransform, Z.shape[0], Z.shape[1], make_grid=True)
spac = np.sqrt(np.sum(np.asarray(geoTransform[1:3])**2))

import velocity products

In [8]:
V_x = read_geo_image(os.path.join(dat_dir, vel_name + v_x_name))[0]
V_y = read_geo_image(os.path.join(dat_dir, vel_name + v_y_name))[0]
V_x_std = read_geo_image(os.path.join(dat_dir, vel_name + v_y_std))[0]
V_y_std = read_geo_image(os.path.join(dat_dir, vel_name + v_y_std))[0]
V_count = read_geo_image(os.path.join(dat_dir, vel_name + v_count))[0]

NaN_val = V_x[0][0]
V_x[V_x==NaN_val], V_y[V_y==NaN_val] = np.nan, np.nan
V_x_std[V_x_std==NaN_val], V_y_std[V_y_std==NaN_val] = np.nan, np.nan

make topographic gradient

In [9]:
fx,fy = get_grad_filters(ftype='kroon', tsize=3, order=1)
Z_x, Z_y = ndimage.convolve(Z, fx), ndimage.convolve(Z, fy)

filling of the velocity field

In [10]:
V_x_fill = local_infilling_filter(V_x, tsize=5)
V_y_fill = local_infilling_filter(V_y, tsize=5)

cleaning of the velocity field

In [11]:
V_clean = anistropic_diffusion_scalar(np.dstack((V_x_fill, V_y_fill)), iter=1)

d8 implementation

In [13]:
RGI_new = d8_flow(RGI, -1*V_clean[:,:,0], +1*V_clean[:,:,1], iter=10)